# Real-time Video

In [ ]:
import cv2

inputVideo = cv2.VideoCapture(0)
#print(type(inputVideo))
#check if connection with camera is successfully
if inputVideo.isOpened():
    ret, frame = inputVideo.read()  #capture a frame from live video

print(inputVideo.isOpened())

cameraMatrix = None
distCoeffs = None
markerLength = 0.05

# You can read camera parameters from tutorial_camera_params.yml
fs = cv2.FileStorage("tutorial_camera_params.yml", cv2.FILE_STORAGE_READ)
cameraMatrix = fs.getNode("cameraMatrix").mat()
distCoeffs = fs.getNode("distCoeffs").mat()
fs.release()

# Set coordinate system
objPoints = np.array([[-markerLength/2.0, markerLength/2.0, 0], [markerLength/2.0, markerLength/2.0, 0], [markerLength/2.0, -markerLength/2.0, 0], [-markerLength/2.0, -markerLength/2.0, 0]], dtype=np.float32)

detectorParams = cv2.aruco.DetectorParameters()
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

while True:
    ret, frame = inputVideo.read()
    if not ret:
        break
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(image, dictionary, parameters=detectorParams)
    if ids is not None and len(ids) > 0:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, markerLength, cameraMatrix, distCoeffs)
        for rvec, tvec in zip(rvecs, tvecs):
            cv2.drawFrameAxes(frame, cameraMatrix, distCoeffs, rvec, tvec, 0.1)
    cv2.imshow("out", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

inputVideo.release()
cv2.destroyAllWindows()

## Video Input

In [8]:
import cv2
import numpy as np
import pandas as pd

inputVideo = cv2.VideoCapture("video\whiteBorders_Trim.mp4")

if inputVideo.isOpened():
    ret, frame = inputVideo.read()

print(inputVideo.isOpened())

cameraMatrix = None
distCoeffs = None
markerLength = 0.05

fs = cv2.FileStorage("tutorial_camera_params.yml", cv2.FILE_STORAGE_READ)
cameraMatrix = fs.getNode("cameraMatrix").mat()
distCoeffs = fs.getNode("distCoeffs").mat()
fs.release()

detectorParams = cv2.aruco.DetectorParameters()
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

pose_data = pd.DataFrame(columns=['Time', 'Marker ID', 'tx', 'ty', 'tz', 'rx', 'ry', 'rz'])

prev_positions = {}

while True:
    ret, frame = inputVideo.read()
    if not ret:
        break
    height, width = frame.shape[:2]
    scale = 0.5
    scaled_frame = cv2.resize(frame, (int(width*scale), int(height*scale)))
    image = cv2.cvtColor(scaled_frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(image, dictionary, parameters=detectorParams)
    if ids is not None and len(ids) > 0:
        cv2.aruco.drawDetectedMarkers(scaled_frame, corners, ids)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, markerLength, cameraMatrix, distCoeffs)
        detected_ids = [int(i) for i in ids.ravel()]
        if set(detected_ids) == {11, 15, 3} or set(detected_ids) == {14, 9, 7}:
            intersection = np.mean(tvecs, axis=0)
            rvec = np.zeros((3,1))
            tvec = np.zeros((3,1))
            imgpts, _ = cv2.projectPoints(intersection, rvec, tvec, cameraMatrix, distCoeffs)

            # draw circle at intersection point
            center = tuple(map(int, imgpts[0].ravel()))
            cv2.circle(scaled_frame, center, 5, (0, 255, 0), -1)

            # append pose data to dataframe
            data = {'Time': pd.Timestamp.now(), 'Marker ID': 'Intersection', 'tx': intersection[0][0], 'ty': intersection[0][1], 'tz': intersection[0][2], 'rx': 0, 'ry': 0, 'rz': 0}
            pose_data = pose_data.append(data, ignore_index=True)

        else:
            for id, rvec, tvec in zip(ids, rvecs, tvecs):
                cv2.drawFrameAxes(scaled_frame, cameraMatrix, distCoeffs, rvec, tvec, 0.1)
                cv2.aruco.drawDetectedMarkers(scaled_frame, corners, ids)

                # add offset to pose for specific marker IDs
                if int(id) in [10, 8, 6, 13]:
                    cube_width = 0.1
                    tvec = tvec + np.array([[cube_width/2, cube_width/2, 0]]).T

                if tuple(id) in prev_positions:
                    prev_t, prev_p, prev_o = prev_positions[tuple(id)]
                    dt = pd.Timestamp.now() - prev_t
                    v = (tvec - prev_p) #/ dt.total_seconds()
                    p = tvec + v*dt.total_seconds()
                    o = rvec + (rvec - prev_o) # / dt.total_seconds()
                else:
                    p, o = tvec, rvec
                    
                if set([(11, 15, 3)]).issubset(set(map(tuple, ids))) or set([(14, 9, 7)]).issubset(set(map(tuple, ids))):
                    intersection = np.mean(tvecs, axis=0)
                elif set([12, 5]).issubset(set(map(tuple,ids))) or set([6, 13]).issubset(set(map(tuple,ids))):
                    if 12 in ids and 5 in ids:
                        index12 = ids.tolist().index(12)
                        pose_12 = [rvecs[index12], tvecs[index12]]
                        index5 = ids.tolist().index(5)
                        pose_5 = [rvecs[index5], tvecs[index5]]
                    elif 6 in ids and 13 in ids:
                        index6 = ids.tolist().index(6)
                        pose_6 = [rvecs[index6], tvecs[index6]]
                        index13 = ids.tolist().index(13)
                        pose_13 = [rvecs[index13], tvecs[index13]]
                    elif 10 in ids and 8 in ids:
                        index10 = ids.tolist().index(10)
                        pose_10 = [rvecs[index10], tvecs[index10]]
                        index8 = ids.tolist().index(8)
                        pose_8 = [rvecs[index8], tvecs[index8]]
                    elif (11 in ids and 15 in ids) or (11 in ids and 3 in ids) or (15 in ids and 3 in ids):
                        index3 = ids.tolist().index(3)
                        pose_3 = [rvecs[index3], tvecs[index3]]
                        index11 = ids.tolist().index(11)
                        pose_11 = [rvecs[index11], tvecs[index11]]
                        index15 = ids.tolist().index(15)
                        pose_15 = [rvecs[index15], tvecs[index15]]

                    if 12 in ids and 5 in ids:
                        intersection = ((pose_12[1] + pose_5[1]) / 2) #+ ((markerLength * np.array([1, 0, 0])) / 2)
                    elif 6 in ids and 13 in ids:
                        intersection = ((pose_6[1] + pose_13[1]) / 2) #+ ((markerLength * np.array([0, 0, 1])) / 2)
                    elif 10 in ids and 8 in ids:
                        intersection = ((pose_10[1] + pose_8[1]) / 2) #+ ((markerLength * np.array([0, 0, 1])) / 2)
                    elif 11 in ids and 15 in ids:
                        intersection = ((pose_11[1] + pose_15[1]) / 2) #+ ((markerLength * np.array([0, 0, 1])) / 2)
                    elif 11 in ids and 3 in ids:
                        intersection = ((pose_11[1] + pose_3[1]) / 2) #+ ((markerLength * np.array([0, 0, 1])) / 2)
                    elif 15 in ids and 3 in ids:
                        intersection = ((pose_15[1] + pose_3[1]) / 2) #+ ((markerLength * np.array([0, 0, 1])) / 2)

                else:
                    intersection = None

                if intersection is not None:
                    #rvec = np.zeros((3,1))
                    #tvec = np.zeros((3,1))
                    imgpts, _ = cv2.projectPoints(intersection, rvec, tvec, cameraMatrix, distCoeffs)

                    # draw circle at intersection point
                    center = tuple(map(int, imgpts[0].ravel()))
                    cv2.circle(scaled_frame, center, 5, (0, 255, 0), -1)

                data = {'Time': pd.Timestamp.now(), 'Marker ID': int(id), 'tx': p[0][0], 'ty': p[0][1], 'tz': p[0][2], 'rx': o[0][0], 'ry': o[0][1], 'rz': o[0][2]}
                pose_data = pose_data.append(data, ignore_index=True)
                prev_positions[tuple(id)] = (pd.Timestamp.now(), p, o)

    cv2.imshow("out", scaled_frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

pose_data.to_excel("pose_data.xlsx", index=False)
inputVideo.release()
cv2.destroyAllWindows()

True
